In [75]:
from collections import Counter
import logging
import re
import json
import jieba
from tqdm import tqdm

In [78]:
logger = logging.getLogger('les2')
logger.setLevel(logging.INFO)
console_handler = logging.StreamHandler()
# console_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

In [111]:
with open('./data/question.json', 'r', encoding='utf-8') as f:
    train_set = json.loads(f.read())

In [112]:
# 训练集共20000篇文章
len(train_set)

20000

In [85]:
def precision_recall_f1(prediction, ground_truth):
    if not isinstance(prediction, list):
        prediction_tokens = prediction.split()
    else:
        prediction_tokens = prediction
    if not isinstance(ground_truth, list):
        ground_truth_tokens = ground_truth.split()
    else:
        ground_truth_tokens = ground_truth
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0, 0, 0
    p = 1.0 * num_same / len(prediction_tokens)
    r = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * p * r) / (p + r)
    return p, r, f1

In [86]:
def recall(prediction, ground_truth):
    return precision_recall_f1(prediction, ground_truth)[1]


def f1_score(prediction, ground_truth):
    return precision_recall_f1(prediction, ground_truth)[2]

In [87]:
def metric_max_over_ground_truths(metric_fn, prediction, ground_truth):
    score = metric_fn(prediction, ground_truth)
    return score

In [88]:
# 找到最相关的段落和在段落中的位置
def find_fake_answer(sample):
    for a_idx, answer_token in enumerate(sample['questions']):
        most_related_para = -1
        most_related_para_len = 999999
        max_related_score = 0
#         print('a_idx=',a_idx, 'answer_token=',answer_token)
        for p_idx, para_tokens in enumerate(sample['segmented_article_content']):
            related_score = metric_max_over_ground_truths(recall,
                                                          para_tokens,
                                                          answer_token['segmented_answer'])
#             print('p_idx=',p_idx,'related_score=',related_score)
            if related_score > max_related_score \
                    or (related_score == max_related_score
                        and len(para_tokens) < most_related_para_len):
                most_related_para = p_idx
                most_related_para_len = len(para_tokens)
                max_related_score = related_score
        sample['questions'][a_idx]['most_related_para'] = most_related_para
        most_related_para_tokens = sample['segmented_article_content'][most_related_para]
        
        answer_tokens = set(answer_token['segmented_answer'])
        best_match_score = 0
        best_match_span = [-1, -1]
        best_fake_answer = None
        
        for start_tidx in range(len(most_related_para_tokens)):
            if most_related_para_tokens[start_tidx] not in answer_tokens:
                continue
            for end_tidx in range(len(most_related_para_tokens) - 1, start_tidx - 1, -1):
                span_tokens = most_related_para_tokens[start_tidx: end_tidx + 1]
                match_score = metric_max_over_ground_truths(f1_score, span_tokens,
                                                                answer_token['segmented_answer'])
                if match_score == 0:
                    break
                if match_score > best_match_score:
                    best_match_span = [start_tidx, end_tidx]
                    best_match_score = match_score
                    best_fake_answer = ''.join(span_tokens)
        sample['questions'][a_idx]['answer_spans'] = best_match_span
        sample['questions'][a_idx]['fake_answers'] = best_fake_answer
        sample['questions'][a_idx]['match_scores'] = best_match_score
    return sample

In [120]:
def clean_data(sample):
    # 文章内容和标题分段->分词：将标题插入到分段后的首位置
    sample['segmented_article_title'] = \
        list(jieba.cut(''.join(re.split(r'\u3000+|\s+',sample['article_title'].strip()))))
    
    sample_splited_para = re.split(r'\u3000+|\s{4,10}',sample['article_content'].strip())
    if len(sample_splited_para) == 1 and len(sample_splited_para[0]) > 200:
        sample_splited_para = re.split(r'\。',sample['article_content'].strip())
    sample_splited_list = []
    for para in sample_splited_para:
        sample_splited_list.append(list(jieba.cut(para.strip(), cut_all=False)))
    sample_splited_list.insert(0, sample['segmented_article_title'])

    sample['segmented_article_content'] = sample_splited_list
       
    # 问题和答案分词处理
    for i,question in enumerate(sample['questions']):
        sample['questions'][i]['segmented_question'] = \
            list(jieba.cut(''.join(question['question'].strip().split('\u3000'))))
        sample['questions'][i]['segmented_answer'] = \
            list(jieba.cut(''.join(question['answer'].strip().split('\u3000'))))
    return sample

In [126]:
train_preprocessd = []
for sample in tqdm(train_set[4000:4100]):
    sample = clean_data(sample)
    train_preprocessd.append(find_fake_answer(sample))

100%|████████████████████████████████████████| 100/100 [00:08<00:00, 11.85it/s]


In [121]:
sample = clean_data(train_set[2859])
sample_1 = find_fake_answer(sample)

In [122]:
train_set[2859]

{'article_content': '习近平总书记口言之、身行之，在中国共产党人如何做人上为我们树立了光辉榜样。我们向习近平总书记学做中国共产党人，要做强于信念、敢于担当、甘于奉献、严于自律的组织人，要做践行中华传统美德、热爱优秀传统文化的中国人，同时还要做丰富生动、有情有义的人。新时代中国共产党人如何“做人”，是习近平新时代中国特色社会主义思想的重要论题，值得我们高度重视，认真学习并践行。“做人”，是个重要的问题“做人”，本质上属于人生哲学问题，但与治国理政紧密相关。一部《论语》几乎句句不离“做人”，同时又被历代圣贤视为治国平天下必读之作，以至于有“半部《论语》治天下”之语。习近平总书记一直十分重视研究和解决“做人”问题，党的十八大以来，从新闻报道中我们读到习近平总书记22次谈“做人”，在习近平总书记公开发表的文章中，有29处用到“做人”这个词。习近平总书记谈做人，有以下特点。一是将“做人”与党的建设紧密结合。2013年6月，习近平总书记指出，要通过“洗澡”，务求“既去灰去泥、放松身心，又舒张毛孔、促进新陈代谢，做到干干净净做事、清清白白做人”。2014年“两会”期间，习近平总书记庄重提出“三严三实”，其中强调：“做人要实，就是要对党、对组织、对人民、对同志忠诚老实，做老实人、说老实话、干老实事，襟怀坦白，公道正派。”2016年初，他针对开展“两学一做”学习教育作出重要指示，强调要“把合格的标尺立起来，把做人做事的底线划出来，把党员的先锋形象树起来”。这些重要论述，充分突显了“做人”在党的建设伟大工程中的地位。二是将“做人”与培育新人紧密结合。习近平总书记多次与青少年谈“做人”问题。2014年5月4日，他在同北京大学师生座谈时，要求青年学生“扎扎实实干事，踏踏实实做人”。2014年9月9日，他在同北京师范大学师生代表座谈时，充满深情地说：“做老师，最好的回报是学生成人成才，桃李满天下。想想无数孩子在自己的教育下学到知识、学会做人、事业有成、生活幸福，那是何等让人舒心、让人骄傲的成就。”2015年6月1日，在会见参加中国少先队七大的全体代表时，他指出，“世界上最难的事情，就是怎样做人、怎样做一个好人”。历次谈话中，通过正确“做人”导向使青少年茁壮成长的拳拳之心溢于言表。三是将“口言之”与“身行之”紧密结合。习近平总书记一贯强调和践行“以身作则”。在党的群众路线教

In [118]:
temp = re.split(r'\u3000+|\s{4,10}', train_set[2859]['article_content'])
print(temp)
print(len(temp))

['习近平总书记口言之、身行之，在中国共产党人如何做人上为我们树立了光辉榜样。我们向习近平总书记学做中国共产党人，要做强于信念、敢于担当、甘于奉献、严于自律的组织人，要做践行中华传统美德、热爱优秀传统文化的中国人，同时还要做丰富生动、有情有义的人。新时代中国共产党人如何“做人”，是习近平新时代中国特色社会主义思想的重要论题，值得我们高度重视，认真学习并践行。“做人”，是个重要的问题“做人”，本质上属于人生哲学问题，但与治国理政紧密相关。一部《论语》几乎句句不离“做人”，同时又被历代圣贤视为治国平天下必读之作，以至于有“半部《论语》治天下”之语。习近平总书记一直十分重视研究和解决“做人”问题，党的十八大以来，从新闻报道中我们读到习近平总书记22次谈“做人”，在习近平总书记公开发表的文章中，有29处用到“做人”这个词。习近平总书记谈做人，有以下特点。一是将“做人”与党的建设紧密结合。2013年6月，习近平总书记指出，要通过“洗澡”，务求“既去灰去泥、放松身心，又舒张毛孔、促进新陈代谢，做到干干净净做事、清清白白做人”。2014年“两会”期间，习近平总书记庄重提出“三严三实”，其中强调：“做人要实，就是要对党、对组织、对人民、对同志忠诚老实，做老实人、说老实话、干老实事，襟怀坦白，公道正派。”2016年初，他针对开展“两学一做”学习教育作出重要指示，强调要“把合格的标尺立起来，把做人做事的底线划出来，把党员的先锋形象树起来”。这些重要论述，充分突显了“做人”在党的建设伟大工程中的地位。二是将“做人”与培育新人紧密结合。习近平总书记多次与青少年谈“做人”问题。2014年5月4日，他在同北京大学师生座谈时，要求青年学生“扎扎实实干事，踏踏实实做人”。2014年9月9日，他在同北京师范大学师生代表座谈时，充满深情地说：“做老师，最好的回报是学生成人成才，桃李满天下。想想无数孩子在自己的教育下学到知识、学会做人、事业有成、生活幸福，那是何等让人舒心、让人骄傲的成就。”2015年6月1日，在会见参加中国少先队七大的全体代表时，他指出，“世界上最难的事情，就是怎样做人、怎样做一个好人”。历次谈话中，通过正确“做人”导向使青少年茁壮成长的拳拳之心溢于言表。三是将“口言之”与“身行之”紧密结合。习近平总书记一贯强调和践行“以身作则”。在党的群众路线教育实践活动总结大会上，他说：“正人必先

In [20]:
with open('./data/preprocessed2.json', 'w', encoding='utf-8') as f:
        json.dump([sample_1,sample_1,sample_1], f)

In [80]:
with open('./data/preprocessed_28.json', 'r', encoding='utf-8') as f:
    d = json.load(f)

In [ ]:
class LESDataset(object):
    def __init__(self, train_file, test_file = None):
        self.train_set = self._load_dataset(train_file)
        
        if test_file:
            self.test_set = self._load_dataset(test_file)
    
    def _load_dataset(self, data_path):
        with open(data_path, 'r', encoding='utf8'):
            dataset = json.load(data_path)
        return dataset
    
    def word_iter(self, set_name=None):
        if set_name == 'train':
            dataset = self.train_set
        elif set_name == 'test':
            dataset = self.test